In [1]:
!pip install transformers torch accelerate bitsandbytes
!pip install -U transformers
!pip install -U bitsandbytes
!pip install mypy
!pip install ruff
!pip install bandit

In [2]:
!pip install --upgrade tensorflow
!pip install --upgrade numpy

  Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.0
    Uninstalling numpy-2.2.0:
      Successfully uninstalled numpy-2.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
langchain 0.3.9 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.0.2 which is incompatible.
matplotlib 3.8.0 requires numpy<2,>=1.21, but you have numpy 2.0.2 which is incompatible.
pytensor 2.26.4 requires numpy<2,>=1.17.0, but you have numpy 2.0.2 which is incompatibl

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from google.colab import userdata

In [ ]:
# Get the Hugging Face token from user data
my_key = userdata.get('TOKEN')

# Model identifier
model_id = "meta-llama/Llama-3.2-3B-Instruct"  # Updated model ID

# BitsAndBytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the tokenizer with authentication
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=my_key) # Pass the token here as well

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Load the model with authentication
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    use_auth_token=my_key
)

In [3]:
import numpy as np
import random
import subprocess
import tempfile
import os
import re
import pandas as pd
import subprocess
import tempfile
import matplotlib.pyplot as plt
import ast
import time

In [173]:
class BaseAgent:
    def __init__(self, pipeline, name="BaseAgent"):
        self.pipeline = pipeline
        self.name = name
        self.epsilon = 0.1  # Probabilidade de explorar

        linhas = ["Inicial_C", "Inicial_D", "Processar", "Analisar", "Visualizar", "Estatica", "Executar", "Aprovado"]
        colunas = ["processar_dados", "analisar_dados", "visualizar_resultados", "analise_estatica", "executar_codigo", "propor_refatoracao", "aprovar_codigo"]

        # Inicializando os Q-values como uma matriz de zeros
        matriz = np.zeros((len(linhas), len(colunas)))

        self.q_values = pd.DataFrame(matriz, index=linhas, columns=colunas)

    def choose_action(self, state, actions):
        """
        Escolhe uma ação usando epsilon-greedy, baseado nos Q-values.
        """
        if np.random.rand() < self.epsilon:
            # Escolha aleatória (exploration)
            return random.choice(actions)
        else:
            # Escolha a melhor ação (exploitation)
            valores_selecionados = self.q_values.loc[state][actions]
            max_valor = valores_selecionados.max()
            linhas_maximas = valores_selecionados[valores_selecionados == max_valor].index

            return random.choice(linhas_maximas)


    def update_q_value(self, state, action, reward, next_state, alpha=0.5, gamma=1):
        """
        Atualiza o Q-value da ação escolhida com base na recompensa recebida.
        """
        max_q_next = np.max(self.q_values[next_state])
        self.q_values[state, action] += alpha * (reward + gamma * max_q_next - self.q_values[state, action])

    def generate_response(self, prompt):
        """Gera uma resposta com base no prompt fornecido."""
        response = self.pipeline(prompt)
        return response[0]['generated_text']

    def get_actions(self):
        """
        Retorna uma lista das ações disponíveis para o agente.
        (Implementado nas subclasses)
        """
        raise NotImplementedError


In [174]:
class CodificadorAgent(BaseAgent):
    def __init__(self, pipeline):
        super().__init__(pipeline, name="Codificador")

    def processar_dados(self, state):
        prompt = f"""
        Limpe e prepare os dados com base no problema a seguir:

        Problema: {state['problem']}
        Dados disponíveis: {state.get('dataset', 'Resumo dos dados não fornecido')}
        Feedback: {state.get('feedback', 'Sem feedback disponível')}

        Por favor, escreva apenas um único código Python necessário para realizar essa tarefa.
        """
        return self.generate_response(prompt)

    def analisar_dados(self, state):
        prompt = f"""
        Analise os dados com base no problema a seguir:

        Problema: {state['problem']}
        Código anterior: {state.get('code', 'Nenhum código fornecido')}
        Feedback: {state.get('feedback', 'Sem feedback disponível')}

        Por favor, escreva apenas um único código Python para realizar as análises estatísticas ou construir modelos de aprendizado de máquina.
        """
        return self.generate_response(prompt)

    def visualizar_resultados(self, state):
        prompt = f"""
        Gere visualizações dos dados com base no problema a seguir:

        Problema: {state['problem']}
        Dados disponíveis: {state.get('data_summary', 'Resumo dos dados não fornecido')}
        Feedback: {state.get('feedback', 'Sem feedback disponível')}

        Por favor, escreva apenas um único código Python para criar gráficos ou visualizações relevantes.
        """
        return self.generate_response(prompt)


    def get_actions(self):
        """
        Retorna as ações disponíveis para o Codificador.
        """
        return ['processar_dados', 'analisar_dados', 'visualizar_resultados']


In [175]:
class RevisorAgent(BaseAgent):
    def __init__(self, pipeline):
        super().__init__(pipeline, name="Revisor")

    def analise_estatica(self, state):
        """
        Analisa o código usando Mypy, Ruff e Bandit.
        """
        def run_tool(command):
            try:
                result = subprocess.run(command, capture_output=True, text=True)
                return result.stdout if result.returncode == 0 else result.stderr
            except Exception as e:
                return str(e)

        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as temp_file:
            temp_file.write(state['code'].encode("utf-8"))
            temp_filename = temp_file.name

        mypy_output = run_tool(["mypy", temp_filename])
        ruff_output = run_tool(["ruff", temp_filename])
        bandit_output = run_tool(["bandit", "-r", temp_filename])

        os.remove(temp_filename)

        return f"mypy {mypy_output}, ruff: {ruff_output}, bandit: {bandit_output}"

    def executar_codigo(self, state):
        """
        Avalia o código fornecido, combinando validação de sintaxe e execução.
        Retorna um dicionário com as métricas coletadas.
        """
        def verificar_sintaxe(codigo):
            """
            Verifica se a sintaxe do código é válida.
            Retorna True se válido, caso contrário levanta um SyntaxError.
            """
            try:
                ast.parse(codigo)
                return True
            except SyntaxError as e:
                return f"Erro de sintaxe: {e}"

        def executar_codigo(codigo):
            """
            Executa o código fornecido e mede o tempo de execução.
            Retorna um dicionário com 'tempo_execucao' e 'erro_execucao'.
            """
            resultados = {'tempo_execucao': None, 'erro_execucao': None}
            start_time = time.time()
            try:
                exec(codigo)
                resultados['tempo_execucao'] = time.time() - start_time
            except Exception as e:
                resultados['erro_execucao'] = f"Erro de execução: {e}"
            return resultados

        resultados = {
            'sintaxe_valida': False,
            'erro_execucao': None,
            'tempo_execucao': None
        }

        # Verificar sintaxe
        sintaxe = verificar_sintaxe(state["code"])
        if sintaxe is not True:
            resultados['erro_execucao'] = sintaxe
            return resultados
        resultados['sintaxe_valida'] = sintaxe

        # Executar o código
        exec_result = executar_codigo(state["code"])
        resultados.update(exec_result)

        return resultados

    def propor_refatoracao(self, state):
        prompt = f"""
        Você é um revisor de código. Proponha refatorações para melhorar o código:

        Código:
        {state['code']}
        """
        return self.generate_response(prompt)

    def aprovar_codigo(self, state):
        return "Código Aprovado"

    def get_actions(self):
        """
        Retorna as ações disponíveis para o Revisor.
        """
        return ['analise_estatica', 'executar_codigo', 'propor_refatoracao', 'aprovar_codigo']


In [176]:
class TrainingEnvironment:
    def __init__(self, problems):
        """
        Inicializa o ambiente com uma lista de problemas.
        """
        self.problems = problems
        self.current_problem = None
        self.state = None
        self.joint_rewards = []  # Armazena as recompensas conjuntas

    def reset(self):
        """
        Reinicia o ambiente selecionando um novo problema e reiniciando o estado.
        """
        self.current_problem = random.choice(self.problems)
        self.state = {
            "problem": self.current_problem["description"],
            "dataset": self.current_problem["dataset"],
            "state_C": "Inicial_C",
            "state_R": "Inicial_D",
            "stage": "C",
            "code": "",
            "feedback": "",
            "score": {"execution": 0, "style": 0, "security": 0, "overall": 0},
            "history": ""
        }
        return self.state

    def extract_code(self, output):
        """
        Extrai o código Python delimitado por ```python ... ``` do texto gerado pelo LLM.
        """
        match = re.search(r"```python(.*?)```", output, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            "Nenhum código Python foi encontrado no texto"

    def execute_code(self, code):
        """
        Avalia o código fornecido, combinando validação de sintaxe e execução.
        Retorna um dicionário com as métricas coletadas.
        """
        def verificar_sintaxe(codigo):
            """
            Verifica se a sintaxe do código é válida.
            Retorna True se válido, caso contrário levanta um SyntaxError.
            """
            try:
                ast.parse(codigo)
                return True
            except SyntaxError as e:
                return f"Erro de sintaxe: {e}"

        def executar_codigo(codigo):
            """
            Executa o código fornecido e mede o tempo de execução.
            Retorna um dicionário com 'tempo_execucao' e 'erro_execucao'.
            """
            resultados = {'tempo_execucao': None, 'erro_execucao': None}
            start_time = time.time()
            try:
                exec(codigo)
                resultados['tempo_execucao'] = time.time() - start_time
            except Exception as e:
                resultados['erro_execucao'] = f"Erro de execução: {e}"
            return resultados

        resultados = {
            'sintaxe_valida': False,
            'erro_execucao': None,
            'tempo_execucao': None
        }

        # Verificar sintaxe
        sintaxe = verificar_sintaxe(code)
        if sintaxe is not True:
            resultados['erro_execucao'] = sintaxe
            return resultados
        resultados['sintaxe_valida'] = sintaxe

        # Executar o código
        exec_result = executar_codigo(code)
        resultados.update(exec_result)

        return resultados

    def analyze_code(self, code):
        """
        Analisa o código usando Mypy, Ruff e Bandit.
        """
        def run_tool(command):
            try:
                result = subprocess.run(command, capture_output=True, text=True)
                return result.stdout if result.returncode == 0 else result.stderr
            except Exception as e:
                return str(e)

        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as temp_file:
            temp_file.write(code.encode("utf-8"))
            temp_filename = temp_file.name

        mypy_output = run_tool(["mypy", temp_filename])
        ruff_output = run_tool(["ruff", temp_filename])
        bandit_output = run_tool(["bandit", "-r", temp_filename])

        os.remove(temp_filename)

        return {"mypy": mypy_output, "ruff": ruff_output, "bandit": bandit_output}

    def evaluate_code(self, code):
        """
        Avalia o código gerado e calcula a recompensa conjunta.
        """
        execution_result = self.execute_code(code)
        analysis_result = self.analyze_code(code)

        execution_score = 100 if execution_result["erro_execucao"] is None else -100
        sintaxe_score = 100 if execution_result["sintaxe_valida"] else -100
        time_score = 100 - 1000*execution_result["tempo_execucao"] if execution_result["tempo_execucao"] is not None else 0

        overall_score_exec = (execution_score + sintaxe_score + time_score) / 3

        if re.search(r"no issues found", analysis_result["mypy"].lower()):
            quality_score = 100
        else:
            quality_score = 50
        style_score = 100 if "all checks passed!\n" in analysis_result["ruff"].lower() else 50
        if re.search(r"no issues identified", analysis_result["bandit"].lower().strip()):
            security_score = 100
        else:
            security_score = 50

        overall_score_anas = (quality_score + style_score + security_score) / 3

        overall_score = (overall_score_anas + overall_score_exec)/2 if overall_score_exec > 50 else overall_score_exec

        self.state["code"] = code
        self.state["score"] = {
            "execution": overall_score_exec,
            "quality": quality_score,
            "style": style_score,
            "security": security_score,
            "overall": overall_score,
        }

        joint_reward = overall_score
        return joint_reward

    def step_coder(self, coder_agent):
        """
        Executa o passo do coder no ambiente.
        """
        print("--- Passo do Coder ---")
        self.state["history"] += f"--- Passo do Coder --- \n"

        coder_action = coder_agent.choose_action(self.state['state_C'], coder_agent.get_actions())

        print(f"Ação do Coder: {coder_action}")
        self.state["history"] += f"Ação do Coder: {coder_action} \n"

        if coder_action == 'processar_dados':
            output = coder_agent.processar_dados(self.state)
            self.state["state_C"] = 'Processar'
        elif coder_action == 'analisar_dados':
            output = coder_agent.analisar_dados(self.state)
            self.state["state_C"] = 'Analisar'
        elif coder_action == 'visualizar_resultados':
            output = coder_agent.visualizar_resultados(self.state)
            self.state["state_C"] = 'Visualizar'
        self.state["stage"] = 'R'

        print(f"Output Gerado pelo Coder:\n{output}")
        self.state["history"] += f"Output Gerado pelo Coder:\n{output} \n"

        code = self.extract_code(output) if self.extract_code(output) != "Nenhum código Python foi encontrado no texto" else self.state["code"]
        self.state["code"] = code

        print(f"Código Gerado pelo Coder:\n{code}")
        self.state["history"] += f"Código Gerado pelo Coder:\n{code} \n"


    def step_reviewer(self, reviewer_agent):
        """
        Executa o passo do reviewer no ambiente.
        """
        print("--- Passo do Reviewer ---")
        self.state["history"] += f"--- Passo do Reviewer --- \n"

        reviewer_action = reviewer_agent.choose_action(self.state['state_R'], reviewer_agent.get_actions())

        print(f"Ação do Reviewer: {reviewer_action}")
        self.state["history"] += f"Ação do Reviewer: {reviewer_action} \n"

        if reviewer_action == 'analise_estatica':
            feedback = reviewer_agent.analise_estatica(self.state)
            self.state["state_R"] = 'Estatica'
        elif reviewer_action == 'executar_codigo':
            feedback = reviewer_agent.executar_codigo(self.state)
            self.state["state_R"] = 'Executar'
        elif reviewer_action == 'propor_refatoracao':
            feedback = reviewer_agent.propor_refatoracao(self.state)
            self.state["stage"] = 'C'
        elif reviewer_action == 'aprovar_codigo':
            feedback = reviewer_agent.aprovar_codigo(self.state)
            self.state["state_R"] = 'Aprovado'

        print(f"Feedback do Reviewer:\n{feedback}")
        self.state["history"] += f"{feedback} \n"

    def train_qlearning(self, coder_agent, reviewer_agent, episodes=10, epsilon_decay=0.99, min_epsilon=0.01):
        self.reset()

        for episode in range(episodes):
          print(f"=== Episódio {episode + 1} ===")
          self.state["history"] += f"=== Episódio {episode + 1} ===  \n"
          while True:
            if self.state["state_R"] == "Aprovado":
                joint_reward = self.evaluate_code(self.state["code"])
                print(f"Recompensa Conjunta: {joint_reward} \n")

                self.state["history"] += f"Recompensa Conjunta: {joint_reward}"

                reviewer_agent.update_q_value(reviewer_agent.choose_action(self.state), joint_reward)

                self.joint_rewards.append(joint_reward)

                # Atualiza os epsilon dos agentes
                coder_agent.epsilon = max(min_epsilon, coder_agent.epsilon * epsilon_decay)
                reviewer_agent.epsilon = max(min_epsilon, reviewer_agent.epsilon * epsilon_decay)

                # Exibe o gráfico no final do treinamento
                if episode == episodes - 1:
                    self.plot_rewards()
                self.reset()
                break
            elif self.state["stage"] == "C":
                self.step_coder(coder_agent)
            else:
                self.step_reviewer(reviewer_agent)

    def plot_rewards(self):
        """Plota o gráfico de recompensas acumuladas ao longo dos episódios."""
        plt.figure(figsize=(10, 6))
        plt.plot(self.joint_rewards, label="Recompensas Conjuntas", color="red")
        plt.title("Evolução das Recompensas Durante o Treinamento", fontsize=14)
        plt.xlabel("Episódios", fontsize=12)
        plt.ylabel("Recompensa", fontsize=12)
        plt.legend()
        plt.grid(True)
        plt.show()


In [177]:
from transformers import pipeline

# Criar pipelines fictícios para os agentes
pipeline_codificador = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000, repetition_penalty=1.1)
pipeline_revisor = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000, repetition_penalty=1.1)

# Inicializar agentes
codificador = CodificadorAgent(pipeline_codificador)
revisor = RevisorAgent(pipeline_revisor)


Device set to use cuda:0
Device set to use cuda:0


In [178]:
import pandas as pd

def read_csv(file_path):
    """Lê um arquivo CSV e retorna um DataFrame."""
    try:
        df = pd.read_csv(file_path, sep=';')
        print(f"Arquivo {file_path} carregado com sucesso!")
        print(df.head())  # Mostra as primeiras 5 linhas
        return df
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {e}")
        return None

# Exemplo de uso
file_path = "sales_data.csv"  # Substitua pelo caminho real do seu arquivo
df = read_csv(file_path)

Arquivo sales_data.csv carregado com sucesso!
  Produto,Preço,Quantidade
0         Produto A,10.0,5
1         Produto B,20.5,3
2         Produto C,15.0,8
3         Produto D,25.0,2


In [179]:
# Inicializar ambiente
problems = [
    {"description": "Leia o arquivo 'sales_data.csv' e encontre os valores médio para cada coluna.", "dataset": df}
]
env = TrainingEnvironment(problems)

In [180]:

state = env.reset()
reward_coder = env.train_qlearning(codificador, revisor)
print("Recompensa Codificador:", reward_coder)


=== Episódio 1 ===
--- Passo do Coder ---
Ação do Coder: processar_dados
Output Gerado pelo Coder:

        Limpe e prepare os dados com base no problema a seguir:

        Problema: Leia o arquivo 'sales_data.csv' e encontre os valores médio para cada coluna.
        Dados disponíveis:   Produto,Preço,Quantidade
0         Produto A,10.0,5
1         Produto B,20.5,3
2         Produto C,15.0,8
3         Produto D,25.0,2
        Feedback: 

        Por favor, escreva apenas um único código Python necessário para realizar essa tarefa.
        


```python
import pandas as pd

# Carregar os dados do arquivo CSV
df = pd.read_csv('sales_data.csv')

# Encontrar o valor médio de cada coluna
media_cada_coluna = df.mean()

# Imprimir os resultados
print(media_cada_coluna)
```

Aqui está uma breve explicação do que cada parte de esse código faz:

*   `import pandas as pd`: Importa a biblioteca Pandas e atribui-a como 'pd'.
*   `df = pd.read_csv('sales_data.csv')`: Carrega os dados do arquivo CSV 

TypeError: BaseAgent.choose_action() missing 1 required positional argument: 'actions'

In [181]:
relatorio = env.state["history"]

print(relatorio)

=== Episódio 1 ===  
--- Passo do Coder --- 
Ação do Coder: processar_dados 
Output Gerado pelo Coder:

        Limpe e prepare os dados com base no problema a seguir:

        Problema: Leia o arquivo 'sales_data.csv' e encontre os valores médio para cada coluna.
        Dados disponíveis:   Produto,Preço,Quantidade
0         Produto A,10.0,5
1         Produto B,20.5,3
2         Produto C,15.0,8
3         Produto D,25.0,2
        Feedback: 

        Por favor, escreva apenas um único código Python necessário para realizar essa tarefa.
        


```python
import pandas as pd

# Carregar os dados do arquivo CSV
df = pd.read_csv('sales_data.csv')

# Encontrar o valor médio de cada coluna
media_cada_coluna = df.mean()

# Imprimir os resultados
print(media_cada_coluna)
```

Aqui está uma breve explicação do que cada parte de esse código faz:

*   `import pandas as pd`: Importa a biblioteca Pandas e atribui-a como 'pd'.
*   `df = pd.read_csv('sales_data.csv')`: Carrega os dados do arquivo 

In [182]:
revisor.executar_codigo(env.state)

           Preço  Quantidade
count   4.000000    4.000000
mean   17.625000    4.500000
std     6.523994    2.645751
min    10.000000    2.000000
25%    13.750000    2.750000
50%    17.750000    4.000000
75%    21.625000    5.750000
max    25.000000    8.000000


{'sintaxe_valida': True,
 'erro_execucao': None,
 'tempo_execucao': 0.0077838897705078125}

In [159]:
import pandas as pd

# Carregar os dados do CSV
df = pd.read_csv('sales_data.csv')

# Obter os valores médios de cada coluna
media_valores = df.mean()

# Imprimir os resultados
print(media_valores)

TypeError: Could not convert ['Produto AProduto BProduto CProduto D'] to numeric

In [106]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1000,
    repetition_penalty=1.2,
    top_k=50,
    top_p=0.9,
    truncation=True,
    batch_size=4
)


Device set to use cuda:0


In [107]:
def resposta(prompt):
    response = pipe(prompt)
    return response[0]['generated_text']

In [108]:
prompt = f"""
Pontue o seguinte histórico com base nos critérios abaixo e retorne um resumo no formato especificado:

Critérios:
Descrição do Problema:
- Clareza: A descrição do problema é clara e compreensível? [10 pontos máximo]
- Acurácia: A descrição do problema é precisa e relevante? [10 pontos máximo]

Descrição dos Dados:
- Completude: Todos os conjuntos de dados relevantes são descritos com detalhes suficientes? [10 pontos máximo]
- Análise de Qualidade de Dados: A qualidade e as características dos dados são discutidas (por exemplo, valores ausentes, outliers)? [10 pontos máximo]
- Visualização: Os dados são bem visualizados usando gráficos ou tabelas? [10 pontos máximo]

Metodologia:
- Abordagem: A metodologia escolhida é adequada para resolver o problema de análise de dados? [10 pontos máximo]
- Justificativa: Há uma justificativa clara para o motivo pelo qual métodos ou técnicas específicas foram escolhidos? [10 pontos máximo]
- Implementação: A implementação da metodologia é descrita com precisão e detalhes? [10 pontos máximo]

Resultados:
- Precisão: Os resultados são precisos e consistentes com os objetivos da análise? [10 pontos máximo]
- Entendimento: Os resultados são interpretados e discutidos adequadamente? [10 pontos máximo]
- Visualização: Os resultados são visualizados de forma eficaz e fáceis de entender (por exemplo, gráficos, tabelas)? [10 pontos máximo]

Conclusão:
- Resumo: A conclusão resume sucintamente as principais descobertas do relatório? [10 pontos máximo]
- Implicações: As implicações dos resultados são discutidas? [10 pontos máximo]
- Recomendações: Há recomendações acionáveis ​​ou próximas etapas? [10 pontos máximo]

Formato de saída esperado:
Nota para cada critério (em forma de tabela):
Clareza: X/10
Acurácia: X/10
Completude: X/10
Análise de Qualidade de Dados: X/10
Visualização (Dados): X/10
Abordagem: X/10
Justificativa: X/10
Implementação: X/10
Precisão: X/10
Entendimento: X/10
Visualização (Resultados): X/10
Resumo: X/10
Implicações: X/10
Recomendações: X/10

Nota Final: XX/150

Histórico:
{relatorio}
"""


print(resposta(prompt))


Pontue o seguinte histórico com base nos critérios abaixo e retorne um resumo no formato especificado:

Critérios:
Descrição do Problema:
- Clareza: A descrição do problema é clara e compreensível? [10 pontos máximo]
- Acurácia: A descrição do problema é precisa e relevante? [10 pontos máximo]

Descrição dos Dados:
- Completude: Todos os conjuntos de dados relevantes são descritos com detalhes suficientes? [10 pontos máximo]
- Análise de Qualidade de Dados: A qualidade e as características dos dados são discutidas (por exemplo, valores ausentes, outliers)? [10 pontos máximo]
- Visualização: Os dados são bem visualizados usando gráficos ou tabelas? [10 pontos máximo]

Metodologia:
- Abordagem: A metodologia escolhida é adequada para resolver o problema de análise de dados? [10 pontos máximo]
- Justificativa: Há uma justificativa clara para o motivo pelo qual métodos ou técnicas específicas foram escolhidos? [10 pontos máximo]
- Implementação: A implementação da metodologia é descrita c

In [1]:
from typing import Dict, List

def pontuar_relatorio(relatorio: str) -> Dict[str, int]:
    """
    Avalia um relatório com base nos critérios definidos e atribui pontuações.

    Parâmetros:
        relatorio (str): O relatório a ser avaliado.

    Retorna:
        Dict[str, int]: Dicionário com as pontuações para cada critério e a nota total.
    """
    criterios = {
        "Descrição do Problema - Clareza": ["problema", "dados", "claro", "tarefa"],
        "Descrição do Problema - Acurácia": ["preciso", "relevante", "específico"],
        "Descrição dos Dados - Completude": ["dados disponíveis", "informações fornecidas"],
        "Descrição dos Dados - Qualidade": ["valores ausentes", "outliers", "valores extremos"],
        "Descrição dos Dados - Visualização": ["gráfico", "visualizar", "tabela"],
        "Metodologia - Abordagem": ["código", "procedimento", "análise"],
        "Metodologia - Justificativa": ["escolha", "motivo", "explicação"],
        "Metodologia - Implementação": ["implementar", "executar", "detalhar"],
        "Resultados - Precisão": ["preciso", "correto", "média"],
        "Resultados - Entendimento": ["interpretar", "discutir", "resultado"],
        "Resultados - Visualização": ["gráfico", "visualização", "figura"],
        "Conclusão - Resumo": ["resumo", "conclusão", "finalizar"],
        "Conclusão - Implicações": ["impacto", "implicação", "efeito"],
        "Conclusão - Recomendações": ["sugestão", "próximos passos", "melhoria"]
    }

    pontuacoes = {criterio: 0 for criterio in criterios}

    for criterio, palavras_chave in criterios.items():
        for palavra in palavras_chave:
            if palavra.lower() in relatorio.lower():
                pontuacoes[criterio] += 2  # Atribuir pontos para cada palavra-chave encontrada

        pontuacoes[criterio] = min(pontuacoes[criterio], 10)  # Limitar pontuação máxima por critério

    pontuacoes["Nota Total"] = sum(pontuacoes.values())
    return pontuacoes

# Avaliar o relatório
resultado_pontuacao = pontuar_relatorio(relatorio)

# Exibir as pontuações
for criterio, nota in resultado_pontuacao.items():
    print(f"{criterio}: {nota}")


NameError: name 'relatorio' is not defined

In [81]:
!pip install pandas


In [12]:
import pandas as pd

# Dados para o CSV
data = {
    "Produto": ["Produto A", "Produto B", "Produto C", "Produto D"],
    "Preço": [10.0, 20.5, 15.0, 25.0],
    "Quantidade": [5, 3, 8, 2],
}

# Criando o DataFrame
df = pd.DataFrame(data)

# Salvando como CSV
file_path = "sales_data.csv"
df.to_csv(file_path, index=False)
print(f"Arquivo {file_path} criado com sucesso!")


Arquivo sales_data.csv criado com sucesso!


In [144]:
import pandas as pd

def read_csv(file_path):
    """Lê um arquivo CSV e retorna um DataFrame."""
    try:
        df = pd.read_csv(file_path)
        print(f"Arquivo {file_path} carregado com sucesso!")
        print(df.head())  # Mostra as primeiras 5 linhas
        return df
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {e}")
        return None

# Exemplo de uso
file_path = "sales_data.csv"  # Substitua pelo caminho real do seu arquivo
df = read_csv(file_path)


Arquivo sales_data.csv carregado com sucesso!
     Produto  Preço  Quantidade
0  Produto A   10.0           5
1  Produto B   20.5           3
2  Produto C   15.0           8
3  Produto D   25.0           2


In [11]:
import pandas as pd

def load_and_format_csv(file_path, max_rows=5):
    """Carrega um CSV e o formata como texto para incluir no prompt."""
    try:
        df = pd.read_csv(file_path)
        preview = df.head(max_rows).to_string(index=False)
        print("Pré-visualização do CSV para o prompt:")
        print(preview)
        return preview
    except Exception as e:
        print(f"Erro ao carregar o CSV: {e}")
        return None

# Exemplo de uso
file_path = "sales_data.csv"
csv_preview = load_and_format_csv(file_path)


Erro ao carregar o CSV: [Errno 2] No such file or directory: 'sales_data.csv'


In [ ]:
def generate_code(self, prompt):
    response = self.pipeline(prompt)
    return response[0]["generated_text"]

In [ ]:
if csv_preview:  # Certifique-se de que o arquivo foi carregado corretamente
    prompt = f"""
    Você é um especialista em análise de dados. Abaixo está uma prévia do arquivo 'sales_data.csv':

    {csv_preview}

    Sua tarefa é escrever um script Python que:
    1. Adicione uma nova coluna chamada 'Total', que seja o produto de 'Preço' e 'Quantidade'.
    2. Salve o resultado em um arquivo CSV chamado 'processed_sales_data.csv'.

    Por favor, escreva apenas o código Python necessário para realizar essa tarefa.
    """

    # Enviar ao modelo
    response = pipe(prompt)
    print("Resposta do agente:")
    print(response[0]['generated_text'])


KeyboardInterrupt: 

In [ ]:
output = """
```python
import pandas as pd

# Carregar os dados do arquivo CSV
df = pd.read_csv('sales_data.csv')

# Calcular a coluna Total
df['Total'] = df['Preço'] * df['Quantidade']

# Exibir os dados resultantes
print(df)
```

Seu objetivo é criar um novo campo chamado "Total" que seja o resultado do produto do preço e quantidade. Para isso, você deve utilizar a função `*` para multiplicar os preços e quantidades. Por fim, você deve imprimir os dados resultantes para verificar o resultado.

Agora que você tem o código necessário, é hora de executá-lo e obter os resultados. Para isso, você deve salvar o código em um arquivo `.py` e executá-lo no terminal.

Para obter os resultados, você pode executar o seguinte comando no terminal:
```
python nome_do_arquivo.py
```

Assim, você pode obter os resultados desejados.

Agora que você tem o código e a instrução para executá-lo, é hora de finalizar a tarefa. Não se preocupe, é simples.

O que você gostaria de fazer em seguida?

- Executar o código e obter os resultados.
- Fazer mais algortimos com o código.
- Deixar a tarefa e ir para outra coisa.
- Deixar uma nota para a tarefa.

Se você quiser fazer mais algortimos, aqui estão algumas sugestões:
- Adicionar um novo campo chamado "Margem Líquida" que seja o resultado do preço do produto e da quantidade multiplicados por um fator de margem líquida.
- Criar um novo arquivo CSV com os dados resultantes e salvar em um local diferente.
- Usar o código para calcular o custo total de um produto específico.
- Usar o código para calcular o custo total de um grupo de produtos.

Se você quiser adicionar um novo campo, basta utilizar a função `*` para multiplicar os preços e quantidades. Por exemplo, se você quiser criar um novo campo chamado "Margem Líquida", você pode utilizar a seguinte linha de código:
```python
df['Margem Líquida'] = df['Preço'] * df['Quantidade'] * 0.2
```
Ou se você quiser criar um novo campo chamado "Custo Total", você pode utilizar a seguinte linha de código:
```python
df['Custo Total'] = df['Preço'] * df['Quantidade']
```
Lembre-se de que a função `*` é usada para multiplicar os preços e quantidades.

Se você quiser criar um novo arquivo CSV com os dados resultantes, você pode utilizar a seguinte linha de código:
```python
df.to_csv('resultados.csv', index=False)
```
Ou se você quiser criar um novo arquivo CSV com os dados resultantes e salvar em um local diferente, você pode utilizar a seguinte linha de código:
```python
df.to_csv('resultados.csv', index=False, path='/local/desenvolvimento/dados')
```
Lembre-se de que o `path` deve ser um caminho completo até o local desejado.

Se você quiser usar o código para calcular o custo total de um produto específico, você pode utilizar a seguinte linha de código:
```python
custo_total = df.loc[df['Produto'] == 'Produto A', 'Total'].values[0]
```
Ou se você quiser usar o código para calcular o custo total de um grupo de produtos, você pode utilizar a seguinte linha de código:
```python
custo_total = df.loc[df['Produto'] == 'Produto A', 'Total'].values[0]
df['Custo Total'] = df['Preço'] * df['Quantidade']
```
Lembre-se de que a função `loc` é usada para selecionar os dados específicos.

Espero que essas sugestões ajudem! Se você tiver mais algortimos, sinta-se à vontade para compartilhar!

Agora que você tem as sugestões, é hora de escolher o que você gostaria de fazer em seguida.

- Você gostaria de executar o código e obter os resultados.
- Você gostaria de adicionar mais algortimos com o código.
- Você gostaria de deixar a tarefa e ir para outra coisa.
- Você gostaria de deixar uma nota para a tarefa.

Se você quiser executar o código e obter os resultados, basta executar
import pandas as pd

# Carregar os dados do arquivo CSV
df = pd.read_csv('sales_data.csv')

# Calcular a coluna Total
df['Total'] = df['Preço'] * df['Quantidade']

# Exibir os dados resultantes
print(df)
analise_estatica
"""
output

'\n```python\nimport pandas as pd\n\n# Carregar os dados do arquivo CSV\ndf = pd.read_csv(\'sales_data.csv\')\n\n# Calcular a coluna Total\ndf[\'Total\'] = df[\'Preço\'] * df[\'Quantidade\']\n\n# Exibir os dados resultantes\nprint(df)\n```\n\nSeu objetivo é criar um novo campo chamado "Total" que seja o resultado do produto do preço e quantidade. Para isso, você deve utilizar a função `*` para multiplicar os preços e quantidades. Por fim, você deve imprimir os dados resultantes para verificar o resultado. \n\nAgora que você tem o código necessário, é hora de executá-lo e obter os resultados. Para isso, você deve salvar o código em um arquivo `.py` e executá-lo no terminal. \n\nPara obter os resultados, você pode executar o seguinte comando no terminal:\n```\npython nome_do_arquivo.py\n```\n\nAssim, você pode obter os resultados desejados. \n\nAgora que você tem o código e a instrução para executá-lo, é hora de finalizar a tarefa. Não se preocupe, é simples. \n\nO que você gostaria de f

In [ ]:
print(env.state['code'])

import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv('sales_data.csv')

# Calcular a coluna Total
df['Total'] = df['Preço'] * df['Quantidade']

# Exibir o resultado
print(df)
